In [20]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
path = r'C:\Users\fajar\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB045804,Karma Resorts,Other,Past Guests,Karma Bavaria,PF293481,Mrs,Somrudee,Dizon,Female,...,2024-12-01 00:00:00,4,WYNDHAM,WYNDHAM,WYNDHAM,3,0,0,Email and Phone,NO
1,KK062989,Karma Resorts,Other,Past Guests,Karma Kandara,PF061549,Mr,Ryan,Tan,Male,...,2024-12-03 00:00:00,5,HOUSE_USE,V_OWNER,COMP,3,0,0,Email and Phone,NO
2,KSH020197,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF244311,Mr,Billy,Bud'S Son,Male,...,2024-12-03 00:00:00,3,DIR,FF,RETAIL,2,0,0,Email and Phone,NO
3,KSH020907,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF292073,Mrs,Chris,Surman,Unknown,...,2024-12-03 00:00:00,2,DIR,FF,RETAIL,2,0,0,Email and Phone,NO
4,KLM007502,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF295131,Mr,James,Lind,Unknown,...,2024-12-03 00:00:00,1,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
5,KSH020906,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF292072,Mrs,Angie,Clark,Female,...,2024-12-03 00:00:00,1,DIR,FF,RETAIL,2,0,0,Email and Phone,NO
6,KLM007352,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF293385,Mr,Oliver,Paul,Unknown,...,2024-12-02 00:00:00,1,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO
7,KLM006965,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF289751,Mrs,Karen,Bissland,Unknown,...,2024-12-02 00:00:00,1,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
8,KB038549,Karma Resorts,Other,Past Guests,Karma Bavaria,PF254958,Herr,Jistel,Hans-Jürgen,Male,...,2024-12-06 21:00:00,0,CB,CB,GROUP,1,0,0,Email and Phone,NO
9,KB045351,Karma Resorts,Other,Past Guests,Karma Bavaria,PF290977,Herr,Werner,Wiemken,Male,...,2024-12-02 16:45:00,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO


In [22]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [23]:
df_analytic.fillna('empty', inplace=True)

In [24]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [25]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

c:\Users\fajar\Documents\Python\Task Offline\past_guest_toni_app\../zoho_data_app\main_module.py:90: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[df_columns].replace(' ','', regex=True, inplace=True)


In [26]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
20,KSH021366,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF294324,empty,Boom,Karen,Unknown,...,2024-12-01 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
39,KB040425,Karma Resorts,Other,Past Guests,Karma Bavaria,PF080008,empty,Kea,Marketing Gmbh,Unknown,...,2024-12-01 19:30:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
40,KB040194,Karma Resorts,Other,Past Guests,Karma Bavaria,PF262791,Mr,Yuen,Wong,Male,...,2024-12-08 00:00:00,7,WYNDHAM,WYNDHAM,WYNDHAM,4,0,0,Email Only,NO
41,KB045381,Karma Resorts,Other,Past Guests,Karma Bavaria,PF080008,empty,Kea,Marketing Gmbh,Unknown,...,2024-12-01 17:00:00,0,LGR,LGR,GROUP,1,0,0,Email Only,NO
42,KB044518,Karma Resorts,Other,Past Guests,Karma Bavaria,PF286649,Ms,Fiona,Toon,Female,...,2024-12-09 00:00:00,3,WYNDHAM,WYNDHAM,WYNDHAM,4,0,0,Email Only,NO
43,KB044655,Karma Resorts,Other,Past Guests,Karma Bavaria,PF287371,Mr,Ian,Smith,Male,...,2024-12-06 00:00:00,5,WYNDHAM,WYNDHAM,WYNDHAM,4,0,0,Email Only,NO
47,KSH020638,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF290242,Ms,Hema,Vyas,Female,...,2024-12-06 00:00:00,2,COMP,VIP,COMP,2,0,0,Email Only,NO
48,KSH020639,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF290243,Ms,Tina,Elhage,Female,...,2024-12-06 00:00:00,2,COMP,VIP,COMP,2,0,0,Email Only,NO
49,KB040576,Karma Resorts,Other,Past Guests,Karma Bavaria,empty,Firma,Hospired,Net,Unknown,...,2024-12-04 12:00:00,0,CB,CB,GROUP,1,0,0,Email Only,NO
50,KB040577,Karma Resorts,Other,Past Guests,Karma Bavaria,empty,Firma,Hospired,Net,Unknown,...,2024-12-03 12:00:00,0,CB,CB,GROUP,1,0,0,Email Only,NO


In [27]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB045804,Karma Resorts,Other,Past Guests,Karma Bavaria,PF293481,Mrs,Somrudee,Dizon,Female,...,2024-12-01 00:00:00,4,WYNDHAM,WYNDHAM,WYNDHAM,3,0,0,Email and Phone,NO
1,KK062989,Karma Resorts,Other,Past Guests,Karma Kandara,PF061549,Mr,Ryan,Tan,Male,...,2024-12-03 00:00:00,5,HOUSE_USE,V_OWNER,COMP,3,0,0,Email and Phone,NO
2,KSH020197,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF244311,Mr,Billy,Bud'S Son,Male,...,2024-12-03 00:00:00,3,DIR,FF,RETAIL,2,0,0,Email and Phone,NO
3,KSH020907,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF292073,Mrs,Chris,Surman,Unknown,...,2024-12-03 00:00:00,2,DIR,FF,RETAIL,2,0,0,Email and Phone,NO
4,KLM007502,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF295131,Mr,James,Lind,Unknown,...,2024-12-03 00:00:00,1,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
5,KSH020906,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF292072,Mrs,Angie,Clark,Female,...,2024-12-03 00:00:00,1,DIR,FF,RETAIL,2,0,0,Email and Phone,NO
6,KLM007352,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF293385,Mr,Oliver,Paul,Unknown,...,2024-12-02 00:00:00,1,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO
7,KLM006965,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF289751,Mrs,Karen,Bissland,Unknown,...,2024-12-02 00:00:00,1,DIR,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
8,KB038549,Karma Resorts,Other,Past Guests,Karma Bavaria,PF254958,Herr,Jistel,Hans-Jürgen,Male,...,2024-12-06 21:00:00,0,CB,CB,GROUP,1,0,0,Email and Phone,NO
9,KB045351,Karma Resorts,Other,Past Guests,Karma Bavaria,PF290977,Herr,Werner,Wiemken,Male,...,2024-12-02 16:45:00,0,LGR,LGR,GROUP,1,0,0,Email and Phone,NO


In [28]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [29]:
find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [30]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
11,KK062625,Karma Resorts,Other,Past Guests,Karma Kandara,PF290135,Mr,N Vijayandra,Rao,Male,...,2024-12-04 00:00:00,3,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
12,KK059561,Karma Resorts,Other,Past Guests,Karma Kandara,PF283192,Mr,Ramchandran,Menon,Male,...,2024-12-01 00:00:00,7,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
20,KK060781,Karma Resorts,Other,Past Guests,Karma Kandara,PF077919,Mr,Bryan John,Dyer,Male,...,2024-12-01 00:00:00,6,HOUSE_USE,V_OWNER,COMP,8,0,0,Email and Phone,NO
32,KK060420,Karma Resorts,Other,Past Guests,Karma Kandara,PF112216,Mrs,Marina,Voloshenko,Female,...,2024-12-06 00:00:00,26,HOUSE_USE,V_OWNER,COMP,2,0,0,Email and Phone,NO


In [31]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [32]:
count_data_akhir = df_analytic.shape[0]

In [33]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB045804,Karma Resorts,Other,Past Guests,Karma Bavaria,PF293481,Mrs,Somrudee,Dizon,Female,...,2024-12-01 00:00:00,4,WYNDHAM,WYNDHAM,WYNDHAM,3,0,0,Email and Phone,NO
1,KK064548,Karma Resorts,Other,Past Guests,Karma Kandara,PF171603,Mr,Beau,Whittington,Male,...,2024-12-01 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
2,KB039542,Karma Resorts,Other,Past Guests,Karma Bavaria,PF289197,Mr,Giorgos,Gianakakis,Male,...,2024-12-01 00:00:00,32,STAFF_PRIV,STAFF,COMP,1,0,0,Email Only,NO
3,KK064196,Karma Resorts,Other,Past Guests,Karma Kandara,PF293210,Mr,Eassam,Mohammed,Male,...,2024-12-01 00:00:00,5,DIGITAL,GHPA,DIGITAL,2,0,0,Email and Phone,NO
4,KK065049,Karma Resorts,Other,Past Guests,Karma Kandara,PF050744,Mr,Brian,Bowden,Male,...,2024-12-01 00:00:00,3,HOUSE_USE,V_OWNER_NRB,COMP,1,0,0,Email and Phone,NO
5,KB043753,Karma Resorts,Other,Past Guests,Karma Bavaria,PF281635,Frau,Kathrin,Gress,Female,...,2024-12-01 00:00:00,2,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
6,KB045975,Karma Resorts,Other,Past Guests,Karma Bavaria,PF294526,Herr,Sebastian,Baumann,Male,...,2024-12-01 00:00:00,2,DIR,TEL,RETAIL,2,2,0,Email and Phone,NO
7,KB045824,Karma Resorts,Other,Past Guests,Karma Bavaria,PF293738,Dr,Manuel,Schneckenburger,Male,...,2024-12-01 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
8,KB043947,Karma Resorts,Other,Past Guests,Karma Bavaria,PF283076,Frau,Sophia,Plötz,Female,...,2024-12-01 00:00:00,2,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
9,KB046055,Karma Resorts,Other,Past Guests,Karma Bavaria,PF294627,empty,Matthias,Haselbock,Unknown,...,2024-12-01 00:00:00,2,OTA,OTA,RETAIL,2,3,0,Email and Phone,NO


In [34]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [35]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


C:\Users\fajar\AppData\Local\Temp\ipykernel_12188\1513714307.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_analytic['DOB'].replace('empty', '', inplace=True)


In [36]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KB045804,Karma Resorts,Other,Past Guests,Karma Bavaria,PF293481,Mrs,Somrudee,Dizon,Female,...,2024-12-01,4,WYNDHAM,WYNDHAM,WYNDHAM,3,0,0,Email and Phone,NO
1,KK063053,Karma Resorts,Other,Past Guests,Karma Kandara,PF290930,Mr,Rifan,Herriyadi,Male,...,2024-12-01,3,HOUSE_USE,V_OWNER,COMP,2,0,0,Email and Phone,NO
2,KK065187,Karma Resorts,Other,Past Guests,Karma Kandara,PF295059,Mr,.,Nanang,Male,...,2024-12-01,1,COMP,VIP,COMP,2,0,0,Email and Phone,NO
3,KSH021538,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF295164,Mr,Daniel,Jansepar,Unknown,...,2024-12-01,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
4,KK064829,Karma Resorts,Other,Past Guests,Karma Kandara,PF294459,Ms,Charlotte,Mcgoldrick,Female,...,2024-12-01,4,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
5,KSH021366,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF294324,,Boom,Karen,Unknown,...,2024-12-01,1,OTA,OTA,RETAIL,1,0,0,Email and Phone,NO
6,KSH021422,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF294628,,Terri,James,Unknown,...,2024-12-01,1,OTA,OTA,RETAIL,2,0,0,Email and Phone,NO
7,KB044203,Karma Resorts,Other,Past Guests,Karma Bavaria,PF284640,,Casper,Sorensen,Unknown,...,2024-12-01,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
8,KB046038,Karma Resorts,Other,Past Guests,Karma Bavaria,PF294584,Herr,Patrick,Hoffmann,Male,...,2024-12-01,1,OTA,OTA,RETAIL,2,2,0,Email and Phone,NO
9,KB044279,Karma Resorts,Other,Past Guests,Karma Bavaria,PF285090,Frau,Dorothea,Wenderoth,Female,...,2024-12-01,2,DIR,WEBSITE,DIGITAL,6,0,0,Email and Phone,NO


In [37]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 58
Jumlah Data Yang Duplicate : 17
Jumlah Data Setelah Hapus Duplicate : 41
Jumlah Data Yang Termasuk Member: 4
Jumlah Data Setelah Hapus Member : 37
Jumlah Data Akhir : 37


In [38]:
df_analytic.to_excel(r'C:\Users\fajar\Documents\Python\Data\final_analytics.xlsx', index=False)